## Lab 5: AgentCore Evaluations - 고객 지원 에이전트를 위한 온라인 평가

### 개요

이 랩에서는 AgentCore Evaluations를 사용하여 Lab 4의 프로덕션 고객 지원 에이전트를 지속적으로 모니터링하는 방법을 보여줍니다. 고객이 에이전트와 상호 작용할 때 에이전트 성능을 실시간으로 자동 평가하도록 온라인 평가를 구성합니다.

**워크샵 여정:**

- **Lab 1 (완료):** 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **Lab 2 (완료):** 메모리 기능 강화 - 대화 맥락 및 개인화 추가
- **Lab 3 (완료):** 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구를 안전하게 공유
- **Lab 4 (완료):** 프로덕션 배포 - 관측 기능을 갖춘 AgentCore 런타임 사용
- **Lab 5 (현재):** 에이전트 성능 평가 - 온라인 평가로 품질 모니터링
- **Lab 6:** 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성

### 배울 내용

기본 제공 평가자를 사용하여 온라인 평가를 구성하고, 테스트 상호 작용을 생성하고, AgentCore Observability 대시보드를 통해 품질 메트릭을 분석하여 에이전트 성능을 개선하는 방법을 배웁니다.

### 온라인 평가 개요

온라인 평가는 특정 선택된 상호 작용을 분석하는 온디맨드 평가와 달리 프로덕션에 배포된 에이전트를 지속적으로 모니터링합니다. 구성 가능한 규칙이 있는 세션 샘플링, 여러 평가 방법(기본 제공 또는 사용자 지정 평가자), 품질 추세 및 낮은 점수 세션 조사가 포함된 대시보드를 통한 모니터링의 세 가지 구성 요소로 되어 있습니다.

에이전트가 AgentCore 런타임에서 실행되므로, AgentCore Observability는 코드를 자동으로 계측하고 [OTEL](https://opentelemetry.io/) 계측을 사용하여 포괄적인 로그 및 추적을 제공합니다.

### 사전 필수 조건

Lab 4를 완료하여 고객 지원 에이전트가 배포되어 있어야 합니다. Evaluations 권한이 있는 Amazon Bedrock AgentCore에 대한 AWS 계정 액세스 권한이 필요합니다.

### 아키텍처
<div style="text-align:left">
    <img src="images/architecture_lab5_evaluation.png" width="75%"/>
</div>

*온라인 평가는 에이전트 상호 작용을 자동으로 모니터링하고, 샘플링 규칙에 따라 평가자를 적용하고, 분석을 위해 결과를 CloudWatch로 출력합니다.*

### 단계 1: 필요한 라이브러리 가져오기 및 클라이언트 초기화

In [ ]:
from bedrock_agentcore_starter_toolkit import Evaluation, Runtime
import json
import uuid
from pathlib import Path
from boto3.session import Session
from IPython.display import Markdown, display
from lab_helpers.utils import get_ssm_parameter, get_or_create_cognito_pool

In [ ]:
boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

In [ ]:
eval_client = Evaluation(region=region)
runtime_client = Runtime()

### 단계 2: Lab 4에서 에이전트 정보 검색하기

Lab 4 배포 중 저장된 SSM Parameter Store에서 고객 지원 에이전트 ARN을 검색합니다.

In [ ]:
try:
    # Get agent ARN from SSM parameter store (saved in Lab 4)
    agent_arn = get_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
    
    # Extract agent ID from ARN
    agent_id = agent_arn.split(":")[-1].split("/")[-1]
    
    # Set runtime client config path
    runtime_client._config_path = Path.cwd() / ".bedrock_agentcore.yaml"
    
    print("Agent ID:", agent_id)
    print("Agent ARN:", agent_arn)
except Exception as e:
    raise Exception(f"""Missing agent information from Lab 4. Please run lab-04-agentcore-runtime.ipynb first. Error: {str(e)}""")

### 단계 3: 온라인 평가 구성 생성하기

이제 고객 지원 에이전트에 대한 온라인 평가 구성을 생성해 보겠습니다. 기본 제공 평가자를 사용하여 에이전트 성능의 다양한 측면을 평가합니다:

- **Builtin.GoalSuccessRate** - 에이전트가 사용자 목표를 달성하는 정도를 측정
- **Builtin.Correctness** - 응답의 사실적 정확성 평가
- **Builtin.ToolSelectionAccuracy** - 적절한 도구 선택 평가

데모 목적으로 샘플링 비율을 100%로 설정하지만, 프로덕션에서는 트래픽 양에 따라 더 낮은 비율(예: 10-20%)을 사용할 수 있습니다.

In [ ]:
response = eval_client.create_online_config(
    agent_id=agent_id,
    config_name="customer_support_agent_eval",
    sampling_rate=100,  # Evaluate 100% of sessions for demo
    evaluator_list=[
        "Builtin.GoalSuccessRate", 
        "Builtin.Correctness",
        "Builtin.ToolSelectionAccuracy"
    ],
    config_description="Customer support agent online evaluation",
    auto_create_execution_role=True
)

print("Online evaluation configuration created successfully!")
print(f"Configuration ID: {response['onlineEvaluationConfigId']}")

### 단계 4: 구성 상태 확인하기

세부 정보를 검색하여 평가 구성이 올바르게 생성되고 활성화되었는지 확인합니다.

In [ ]:
config_details = eval_client.get_online_config(config_id=response['onlineEvaluationConfigId'])
print("Configuration Details:")
print(json.dumps(config_details, indent=2, default=str))

### 단계 5: 테스트 상호 작용 생성하기

평가를 위한 추적을 생성하기 위해 다양한 쿼리로 고객 지원 에이전트를 호출합니다. 다양한 테스트 시나리오는 평가자가 에이전트 성능을 평가하는 방법을 보여줍니다.

In [ ]:
# Get authentication token
access_token = get_or_create_cognito_pool(refresh_token=True)
print(f"Access token obtained: {access_token['bearer_token'][:20]}...")

def invoke_agent_runtime(prompt, session_id=None):
    """Invoke the agent runtime using starter toolkit"""
    if not session_id:
        session_id = str(uuid.uuid4())
    
    response = runtime_client.invoke(
        payload={"prompt": prompt},
        session_id=session_id,
        bearer_token=access_token['bearer_token']
    )
    
    return response, session_id

#### 테스트 시나리오 1: 제품 정보 쿼리

In [ ]:
session1 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "I need information about the Gaming Console Pro. What are its specifications and price?",
    session1
)
print("Customer Query: Product information request")
display(Markdown(response["response"].replace('\\n', '\n')))

#### 테스트 시나리오 2: 기술 지원 요청

In [ ]:
session2 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "My laptop won't start up. Can you help me troubleshoot this issue?",
    session2
)
print("Customer Query: Technical support request")
display(Markdown(response["response"].replace('\\n', '\n')))

#### 테스트 시나리오 3: 반품 정책 문의

In [ ]:
session3 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "I bought a smartphone last week but it's not working properly. What's your return policy?",
    session3
)
print("Customer Query: Return policy inquiry")
display(Markdown(response["response"].replace('\\n', '\n')))

#### 테스트 시나리오 4: 복잡한 다중 도구 쿼리

In [ ]:
session4 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "I need help with my Gaming Console Pro. First, can you tell me about its warranty? Then I need technical support for connection issues.",
    session4
)
print("Customer Query: Complex multi-tool request")
display(Markdown(response["response"].replace('\\n', '\n')))

#### 테스트 시나리오 5: 일반 기능 쿼리

In [ ]:
session5 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "What kind of support can you provide? List all your available tools and capabilities.",
    session5
)
print("Customer Query: Capability inquiry")
display(Markdown(response["response"].replace('\\n', '\n')))

### 단계 6: 평가 결과 모니터링하기

AgentCore Observability 콘솔을 통해 평가 결과를 모니터링합니다. 시스템이 추적을 처리하고 평가자를 적용할 때 결과가 나타나는 데 몇 분 정도 걸릴 수 있습니다.

#### 대시보드 액세스하기

1. [AgentCore Observability 콘솔](https://console.aws.amazon.com/cloudwatch/home#gen-ai-observability/agent-core/agents)로 이동합니다.
2. 에이전트 목록에서 고객 지원 에이전트를 찾습니다.
3. `DEFAULT` 엔드포인트를 클릭하여 평가 메트릭을 봅니다.
4. 추적 및 세션 보기에서 평가 점수를 찾습니다.

#### 보게 될 내용

대시보드에는 다음이 표시됩니다:
- **목표 성공률(Goal Success Rate)**: 에이전트가 고객 목표를 얼마나 잘 달성하는지
- **정확성(Correctness)**: 제공된 정보의 정확성
- **도구 선택 정확도(Tool Selection Accuracy)**: 쿼리에 대한 적절한 도구 선택

![온라인 평가 대시보드](images/online_evaluations_dashboard.png)

*AgentCore Observability 대시보드에 표시된 평가 메트릭*

### 단계 7: 평가 메트릭 이해하기

**목표 성공률(Goal Success Rate)**은 에이전트가 고객의 주요 의도를 성공적으로 해결했는지 측정합니다. 점수가 높으면 효과적인 문제 해결을 나타내며, 점수가 낮으면 충족되지 않은 요구 사항, 불완전한 응답 또는 잘못 이해된 요청을 나타냅니다.

**정확성(Correctness)**은 응답의 사실적 정확성을 평가합니다. 점수가 높으면 정확하고 신뢰할 수 있는 정보를 나타내며, 점수가 낮으면 잘못된 사실, 오래된 정보 또는 오해의 소지가 있는 지침을 나타냅니다.

**도구 선택 정확도(Tool Selection Accuracy)**는 에이전트가 각 작업에 적절한 도구를 선택하는지 평가합니다. 점수가 높으면 적절한 도구 선택을 나타내며, 점수가 낮으면 잘못된 도구, 불필요한 호출 또는 누락된 도구 사용을 나타냅니다.

### 단계 8: 결과 분석 및 다음 단계

**낮은 목표 성공률의 경우:** 에이전트의 시스템 프롬프트를 개선하고, 도구 설명 및 매개변수를 개선하고, 특정 교육 예제를 추가하세요.

**낮은 정확성 점수의 경우:** 지식 기반을 최신 정보로 업데이트하고, 사실 확인 메커니즘을 개선하고, 도구 응답을 검토하세요.

**도구 관련 문제의 경우:** 도구 매개변수 스키마를 구체화하고, 도구 선택 로직을 개선하고, 도구 설명서를 강화하세요.

**지속적인 모니터링:** 평가 메트릭에 대한 CloudWatch 경보를 설정하고, 추세 분석을 위한 대시보드를 생성하고, 품질 저하에 대한 자동 알림을 구현하세요.

### 단계 9: 정리 (선택 사항)

필요한 경우 아래 코드의 주석 처리를 제거하여 온라인 평가 구성을 비활성화하세요.

In [ ]:
# Uncomment the following lines if you want to disable the evaluation configuration
# eval_client.delete_online_config(config_id=response['onlineEvaluationConfigId'])
# print("Online evaluation configuration disabled")

### 축하합니다! 🎉

**Lab 5: AgentCore Evaluations - 온라인 평가**를 성공적으로 완료했습니다!

### 달성한 내용

목표 성공률(고객 만족 및 문제 해결), 정확성(사실적 정확성) 및 도구 선택 정확도(적절한 도구 사용)를 평가하는 기본 제공 평가자를 사용하여 고객 지원 에이전트에 대한 자동 연속 온라인 평가를 구성했습니다. 평가 결과는 실시간 통찰력을 위해 AgentCore Observability 대시보드와 통합됩니다.

**주요 이점:** 사전 예방적 품질 보증으로 고객에게 영향을 미치기 전에 문제를 포착하고, 데이터 기반 최적화로 개선을 안내하고, 규모에 따른 성능 모니터링을 통해 프로덕션 신뢰도를 높이고, 지속적인 학습으로 패턴과 기회를 식별합니다.

**다음 단계:** 평가 대시보드를 정기적으로 모니터링하고, 품질 임계값에 대한 CloudWatch 경보를 설정하고, 통찰력을 사용하여 에이전트를 반복적으로 개선하고, 도메인별 메트릭에 대한 사용자 지정 평가자 추가를 고려하세요.

### 다음 단계: [Lab 6: 사용자 인터페이스 구축 →](lab-06-frontend.ipynb)

품질이 모니터링되는 에이전트와 고객이 상호 작용할 수 있는 사용자 친화적인 웹 인터페이스를 구축하여 고객 경험을 완성하세요.

이제 고객 지원 에이전트는 포괄적인 품질 모니터링을 갖춘 프로덕션 준비가 되었습니다! 🚀